<a href="https://colab.research.google.com/github/maggiejohnson20/DataProcessingAndVisualization/blob/main/Quiz5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Remember to always authenticate your account!**

In [48]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


To remind me of the data set, I decided to pull the top 15 entries. **Remember : Always remember to leave the big query name at the top.**

In [25]:
%%bigquery --project dataprocessingvisualization

SELECT * 
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
LIMIT 16

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19
5,9900286143,24-Hour Kiosk (Austin B-cycle),924,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,17
6,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6
7,9900286214,Annual Membership (Austin B-cycle),24,2014-10-26 20:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,0
8,9900286540,24-Hour Kiosk (Austin B-cycle),117,2014-10-27 15:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,12
9,13575843,Walk Up,302,2017-01-29 16:42:52+00:00,3464,Pease Park,3464,Pease Park,47


Once we have that, I want to grab the most popular starting points of the race. 

In [36]:
%%bigquery --project dataprocessingvisualization

SELECT COUNT(*) as number_trips, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
ORDER BY number_trips DESC 


,number_trips,start_station_name
0,72799,21st & Speedway @PCL
1,40635,Riverside @ S. Lamar
2,36520,City Hall / Lavaca & 2nd
3,35307,2nd & Congress
4,34758,Rainey St @ Cummings
...,...,...
188,4,Marketing Event
189,2,Eeyore's 2018
190,1,Stolen
191,1,Eeyore's 2017


Now it's time to find the second most popular trip. If you want to find the most popular trip, just remove the "!=" section!

In [45]:
%%bigquery --project dataprocessingvisualization

SELECT start_station_name, COUNT(*) as most_popular_trip
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
HAVING most_popular_trip != (SELECT MAX(most_popular_trip)as maximum_trip
FROM (
SELECT COUNT(*) as most_popular_trip, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
))
ORDER BY most_popular_trip DESC
LIMIT 1

,start_station_name,most_popular_trip
0,Riverside @ S. Lamar,40635


Lastly, I need to find out what trips lasted more than an our long and were also round trips. 

In [47]:
%%bigquery --project dataprocessingvisualization

SELECT *, COUNTIF(duration_minutes>60) as trips_taken_an_hour_plus
FROM (
  SELECT start_station_name, duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = end_station_name
)
GROUP BY duration_minutes, start_station_name
Order BY trips_taken_an_hour_plus DESC


,start_station_name,duration_minutes,trips_taken_an_hour_plus
0,Riverside @ S. Lamar,62,109
1,Riverside @ S. Lamar,61,95
2,Riverside @ S. Lamar,63,94
3,Riverside @ S. Lamar,69,90
4,Zilker Park,61,88
...,...,...,...
24822,26th/Nueces,40,0
24823,26th/Nueces,17,0
24824,26th/Nueces,53,0
24825,26th/Nueces,48,0
